In [9]:
import pandas as pd
import requests
# para extrair informações de páginas HTML
import bs4
from bs4 import BeautifulSoup

# utilizada para nos indicar o caminho do executável do Python
import sys

# Controlar espera entre requisições
import time
# Renderizar HTML
import IPython


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


### Criando um driver

Execute a linha abaixo. Você irá perceber que será criada uma janela do navegador Chrome. Deixe esta janela aberta.

In [11]:
PROXY = "117.250.3.58:8080"
chrome_options = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
#chrome_options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.set_page_load_timeout(60)


### Selecione os aeroportos de origem e destino

In [4]:
aeroporto_origem = 'Curitiba'
aeroporto_destino = 'Fortaleza'

# Começar o Web Scrapping


### Selecionar a url e abrir o site

In [5]:
url = "https://www.skyscanner.com.br/"

# Abrir o site da Decolar
driver.get(url)

### Digitar nossa origem na barra de pesquisa

In [6]:
# Esperar até que a barra de pesquisa esteja disponível
search_bar_origem = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[@id='originInput-input']"))
)


# Limpar o campo de origem e inserir aeroporto origem
#search_bar_origem.clear()
#time.sleep(1)

# Digitar "Curitiba" na caixa de texto
ActionChains(driver).click(search_bar_origem).key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()
ActionChains(driver).send_keys_to_element(search_bar_origem, aeroporto_origem).perform()
time.sleep(1)
ActionChains(driver).send_keys(Keys.ENTER).perform()
time.sleep(1)

### Digitar nosso destino na barra de pesquisa

In [7]:
search_bar_destino = driver.find_element(By.XPATH, "//*[@id='destinationInput-input']")

ActionChains(driver).click(search_bar_destino).key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()
ActionChains(driver).send_keys_to_element(search_bar_destino, aeroporto_destino).perform()
time.sleep(1)
ActionChains(driver).send_keys(Keys.ENTER).perform()
time.sleep(1)

### Selecionar Data de partida e retorno


# Selecionar as datas

In [ ]:
# Inserir a data de partida e retorno

#//*[@id='component-modals']/div[1]/div[2]/div/div[1]/div[3]/div[17] mes 5 dia 17
                                                   #|  1 significa mes atual
#//*[@id="component-modals"]/div[1]/div[2]/div/div[2]/div[3]/div[17]/div mes 6 dia 17
data_partida_selector = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/div[3]/div[1]/div/button/span[2]")
data_partida_selector.click()

In [ ]:
data_partida = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/div[3]/div[2]/div/div/div[1]/div[3]/div[2]/div[1]/div/div[2]/div[3]/div[6]/div/button")
data_partida.click()

In [ ]:
data_retorno_selector = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/div[3]/div[2]/div/button/span[2]")
data_retorno_selector.click()

In [ ]:
data_retorno = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/div[3]/div[2]/div/button/span[2]")
data_retorno.click()

botao_aplicar = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/div[3]/div[3]/div/div/div[1]/div[3]/div[2]/div[2]/div/div[2]/div[4]/div[6]/div/button")
botao_aplicar.click()

Codigo acima funciona! mas precisa ver como fazer para que um input digitado ache a data correta no Xpath

In [ ]:
# Clicar no botão de buscar voos
search_button = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/main/div[1]/div/div[3]/div/button")
search_button.click()

In [ ]:
preco_recomendado = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/div/div[1]/div[3]/div[1]/button/div/div/div/span")
preco_barato = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/div/div[1]/div[3]/div[2]/button/div/div/div/span")
preco_rapido = driver.find_element(By.XPATH, "//*[@id='app-root']/div[1]/div/div/div/div[1]/div[3]/div[3]/button/div/div/div/span")

In [8]:
file_path = 'dados_precos.csv'

def load_existing_data(filepath):
    try:
        # Tenta carregar os dados existentes
        return pd.read_csv(filepath)
    except FileNotFoundError:
        # Se o arquivo não existir, cria um DataFrame vazio com as colunas desejadas
        return pd.DataFrame(columns=['Data', 'Preço Recomendado', 'Preço Mais Barato', 'Preço Mais Rápido'])
    

def save_data(new_data, filepath):
    # Carrega os dados existentes
    df = load_existing_data(filepath)
    
    # Concatena os novos dados
    updated_df = pd.concat([df, new_data], ignore_index=True)
    
    # Salva de volta para o CSV
    updated_df.to_csv(filepath, index=False)

daily_data = pd.DataFrame({
    'Data': pd.Timestamp('today').strftime('%Y-%m-%d'),
    'Preço Recomendado': preco_recomendado,
    'Preço Mais Barato': preco_barato,
    'Preço Mais Rápido': preco_rapido
})

save_data(daily_data, file_path)

NameError: name 'preco_recomendado' is not defined

In [ ]:


# Esperar até que a página de resultados de voos seja carregada
WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@class='resultsWrapper___1i6rS']"))
)

# Obter os resultados dos voos
flight_results = driver.find_elements(By.XPATH, "//div[@class='card___3P3DA']")

# Imprimir os resultados
print("Oportunidades de voos de Curitiba para São Paulo:")
for result in flight_results:
    print(result.text)

# Fechar o navegador
driver.quit()